In [ ]:
# 参考 https://zhuanlan.zhihu.com/p/634169580 理杏仁 年际数据

import requests
import pandas as pd
from pandas import json_normalize

 
#设置自己的Token,下文有说明怎么查看自己的Token
token = "XXX"
 
#设置获取的数据的时间跨度，时间跨度不要超过10年！！！
start_date = '2015-04-30'#起始日期
end_date = '2025-04-30'#结束日期，理杏仁上的财务数据以报告为准，如果一季度报还未发布，那么无法获取到一季度的数据，只能将时间改为上一季度末
 
#设置需要获取的股票代码，统一存入列表，方便提取，不要一次性放入太多股票代码
lst_stock = [
    '002195',
    '002223',
    '002912',
    '300033',
    '601998',
    '605398',
    '688590',
    '002987',
    '301236',
    '300662'
]
 
#上面几项都为准备工作，下面才开始真正设置数据参数
#可根据自己需求，增减下面列表中的参数
#需要获取相应的财务数据,都可在理杏仁上查看，下文会做详细补充
#注意这里使用ｔ。获取累计数据
lst_metrics = [
    'q.cfs.ncffoa.t', # 经营活动产生的现金流量净额
    'q.cfs.ncffia.t', # 投资活动产生的现金流量净额
    'q.cfs.ncfffa.t', # 筹资活动产生的现金流量净额
    "q.m.fcf.t",#自由现金流量
    "q.cfs.cpfpfiaolta.t",#十年资本开支总和
    "q.ps.npadnrpatoshaopc.t", #净利润
    'q.m.m_ds.t', #净营业周期
    'q.ps.oi.t', # 营业收入
    'q.bs.ta.t_y2y',#总资产同比变化
    'q.ps.toi.t_y2y',#营业总收入同比变化
    'q.ps.oc.t',#营业成本
    'q.bs.nraar.t', #应收票据及应收账款
    'q.bs.afc.t', #预收账款
    'q.m.crfscapls_oi_r.t', #销售收现率
    'q.m.wroe.t',#归属于母公司普通股股东的加权ROE
    'q.ps.gp_m.t',#毛利率GM
    'q.m.np_s_r.t',#销售净利率
    'q.m.ta_to.t',#资产周转率
    'q.m.l.t',#杠杆倍数
    'q.bs.lwi_ta_r.t',#有息负债率
    'q.m.tl_ta_r.t',#资产负债率
    'q.m.c_r.t',#流动比率
    'q.m.q_r.t',#速动比率
    'q.ps.npatoshopc.t',#归属于母公司普通股股东的净利润
    'q.bs.tetoshopc.t',#归属于母公司普通股股东权益合计
    'q.bs.tetoshopc_ps.t',#归属于母公司普通股股东的每股股东权益
    'q.ps.npatshaoehopc.t',#归属于母公司股东及其他权益持有者的净利润
    'q.bs.tsc.t',#总股本
    'q.bs.mc.t',#市值
    'q.ps.tp.t',#利润总额
    'q.bs.fa.t',#固定资产
    'q.bs.cip.t',#在建工程
    'q.bs.es.t',#工程物资
    'q.bs.ia.t',#无形资产
    'q.bs.fa.t',#固定资产
    'q.ps.se_r.t',#销售费用率
    'q.ps.ae_r.t',#管理费用率
    'q.ps.rade_r.t',#研发费用率
    'q.ps.fe_r.t',#财务费用率
    'q.bs.d_pe_ttm.t',#PE-TTM(扣非) 
    'q.bs.dyr.t',#股息率
    'q.bs.npaap.t'#应付票据及应付账款
]

# 设置字典列名对应中文解释
# 这里设置的都是上面lst_metrics列表中的数据
# 该步骤是为了后面显示的时候将所有上面财务数据代号改为中文显示，方便阅读
col_map = {
    'name':'公司名称',
    'stockCode':'股票代码',
    'standardDate':'日期',
    'year':'年份',
    'q.ps.oc.t':'营业成本',
    'q.cfs.ncffoa.t': '经营活动产生的现金流量净额',
    'q.cfs.ncffia.t': '投资活动产生的现金流量净额',
    'q.cfs.ncfffa.t': '筹资活动产生的现金流量净额',
    'q.m.fcf.t': '自由现金流',
    'q.cfs.cpfpfiaolta.t': '资本开支',
    'q.ps.npadnrpatoshaopc.t': '净利润',
    'xx_basic':'小熊基本值',
    'xx_increase': '小熊增长值',
    'q.m.m_ds.t': '净营业周期',
    'q.m.crfscapls_oi_r.t': '销售收现率',
    'q.bs.nraar.t':'应收票据及应收账款',
    'q.bs.afc.t': '预收账款',
    'q.ps.oi.t': '营业收入',
    'rec_ratio' : '白条率',
    'adp_ratio' : '预收率',
    'q.m.wroe.t':'归母加权ROE',
    'q.ps.gp_m.t':'毛利率',
    'q.m.np_s_r.t':'销售净利率',
    'q.m.ta_to.t':'资产周转率',
    'q.m.l.t':'杠杆倍数',
    'q.bs.lwi_ta_r.t':'有息负债率',
    'q.m.tl_ta_r.t':'资产负债率',
    'q.m.c_r.t':'流动比率',
    'q.m.q_r.t':'速动比率',
    'q.ps.npatoshopc.t':'归母净利润',
    'q.bs.tetoshopc.t':'净资产',
    'q.bs.tetoshopc_ps.t':'归母权益',
    'q.ps.se_r.t':'销售费用率',
    'q.ps.ae_r.t':'管理费用率',
    'q.ps.rade_r.t':'研发费用率',
    'q.ps.fe_r.t':'财务费用率',
    'q.bs.ta.t_y2y':'总资产同比变化',
    'q.ps.toi.t_y2y':'营业总收入同比变化',
    'q.bs.npaap.t':'应付账款',
    'q.ps.npatshaoehopc.t':'归属于母公司股东及其他权益持有者的净利润',
    'q.bs.tsc.t':'总股本',
    'q.bs.mc.t':'市值',
    'q.bs.fa.t':'固定资产',
    'q.ps.fe_r.t':'财务费用',
    'q.bs.d_pe_ttm.t':'PE-TTM(扣非)' ,
    'q.bs.dyr.t':'股息率',
    'symbol':'现金流肖像'
    
}
 
# 设置字体为黑体，不然制图的时候显示中文会乱码
font_name = 'SimHei'
 
#获取指定财务数据
#本身获取单个数据就不需要用循环，因为获取了多家公司的财务数据，用循环直接合并数据更省心
#这里的参数上面都已经设置好了，只要填入相应位置即可
for i, stock in enumerate(lst_stock):
     # 将所有的参数传入datas
    datas = {
        "token": token,# 上面已经设置好
        "startDate": start_date,# 上面已经设置好
        "endDate": end_date,# 上面已经设置好
        "stockCodes": [stock],# 上面已经设置好
        "metricsList": lst_metrics# 上面已经设置好
        }
        
     #设置好参数，开始获取数据，注意获取的网址！！！
    r = requests.post('https://open.lixinger.com/api/cn/company/fs/non_financial',json = datas)
     
     # 将数据直接转成DataFrame格式
    df_temp = json_normalize(r.json()['data'])
    
    # 合并获取的DataFrame, 第一次获取数据直接存入df_data，后面获取的数据合并到df_data中
    if i == 0:
        df_data = df_temp
    else:
        df_data = pd.concat([df_data, df_temp])
        
#获取公司基本信息
#方法和上面获取财务数据类似，但是需要设置的参数比较少
datas = {
    "token": token,
    "industryType": "non_financial"
}
 
#注意网址，第二步中有讲过
r = requests.post('https://open.lixinger.com/api/a/company',json = datas)
 
df_company = json_normalize(r.json()['data'])
 
#把日期转换成日期数据类型
df_data['standardDate'] = df_data['standardDate'].map(lambda x: x[:10])
df_data['standardDate'] = pd.to_datetime(df_data['standardDate'])
 
#生成年列
df_data['year'] = df_data['standardDate'].map(lambda x: x.year)
 
#只需要年报数据
df_data = df_data[df_data['reportType'] == 'annual_report'].reset_index(drop=True)
#　last_year　是最近一年
last_year = df_data['year'].iloc[0]
 
# first_year是有数据的第一年, 条件判断是看是否有缺少值。
df_test = df_data.loc[:, ['stockCode',
                          'year',
                          'q.m.fcf.t', #自由现金流量
                          'q.ps.npadnrpatoshaopc.t', #归母净利润
                          'q.m.m_ds.t' #净营业周期
 
                      ]]
#去除df_test中有NAN的行
df_test = df_test.dropna(axis=0, how='any')

#查询出 有数据的最早年份
i = 0
list_y = []
while 1:
    if i == df_test.shape[0]-1:
        break
    elif df_test.iat[i,0] != df_test.iat[i+1,0]:
        list_y.append(i)
    i += 1
    
list_y.append(df_test.shape[0]-1)#list_y记录最早年份的位置   
list_firsttyear = []#list_firsttyear取出所有公司的最早年份
 
for each in list_y:
    list_firsttyear.append(df_test.iat[each,1])
first_year = max(list_firsttyear)
 
# 查找df_data中上市时间最短股票的上市年   
df_data = df_data[df_data['year']>= first_year].reset_index(drop=True)
 
 
# 添加一列股票名称　name
df_data = pd.merge(df_data,df_company, on = 'stockCode', how = 'left')
 
 
#判断公司属于重资产还是轻资产=（利润总额/(固定资产+在建工程+工程物资+无形资产)）
df_temp = df_data.loc[:, ['stockCode',
                          'name',
                          'year',
                          'q.ps.tp.t',#利润总额
                          'q.bs.fa.t',#固定资产
                          'q.bs.cip.t',#在建工程
                          'q.bs.es.t',#工程物资
                          'q.bs.ia.t'#无形资产
                          
                         ]]
df_temp['Asset_value'] = df_data['q.ps.tp.t']/(df_data['q.bs.fa.t']+df_data['q.bs.cip.t']+df_data['q.bs.es.t']+df_data['q.bs.ia.t'])
 
def assets_judge(x):
    Asset_type = '重资产'
    if x['Asset_value']>0.1:
        Asset_type = '轻资产'
    return Asset_type
 
df_temp['Asset_type'] = df_temp.apply(assets_judge,axis=1)
df_temp1 = df_temp.pivot(index = 'name', columns = 'year', values = 'Asset_type')
df_temp1.index.name = '公司名称'
df_temp1.columns.name = '年份'
df_temp1

 
#选取需要的列
df_data = df_data.loc[:, ['stockCode',
                          'name',
                          'standardDate',
                          'year',
                          'q.cfs.ncffoa.t',
                          'q.cfs.ncffia.t',
                          'q.cfs.ncfffa.t',
                          'q.m.fcf.t', #自由现金流量
                          'q.cfs.cpfpfiaolta.t', #资本开支
                          'q.ps.npadnrpatoshaopc.t', #归母净利润
                          'q.ps.oi.t',
                          'q.bs.ta.t_y2y',#总资产同比变化
                          'q.ps.toi.t_y2y',#营业总收入同比变化
                          'q.ps.oc.t',#营业成本
                          'q.m.m_ds.t',
                          'q.m.crfscapls_oi_r.t',#销售收现率
                          'q.m.wroe.t',
                          'q.ps.gp_m.t',
                          'q.m.np_s_r.t',
                          'q.m.ta_to.t',
                          'q.m.l.t',
                          'q.bs.lwi_ta_r.t',#有息负债率
                          'q.m.tl_ta_r.t',#资产负债率
                          'q.m.c_r.t',#流动比率
                          'q.m.q_r.t',#速动比率
                          'q.ps.npatoshopc.t',#归属于母公司普通股股东的净利润
                          'q.bs.tetoshopc.t',#归属于母公司普通股股东权益合计
                          'q.bs.tetoshopc_ps.t',#归属于母公司普通股股东的每股股东权益
                          'q.ps.npatshaoehopc.t',#归属于母公司股东及其他权益持有者的净利润
                          'q.bs.tsc.t',#总股本
                          'q.bs.mc.t',#市值
                          'q.bs.fa.t',#固定资产
                          'q.ps.se_r.t',#销售费用
                          'q.ps.ae_r.t',#管理费用
                          'q.ps.rade_r.t',#研发费用
                          'q.ps.fe_r.t',#财务费用
                          'q.bs.d_pe_ttm.t',#PE-TTM(扣非) 
                          'q.bs.dyr.t',#股息率
                          'q.bs.npaap.t'#应付票据及应付账款
                          
                         ]]
 
#初步判断所选股票的自由现金流情况
# 现金流肖像
def judgee(x):
    symbol = '其他'
    if x['q.cfs.ncffoa.t']<0:
        symbol = '危险'   
    elif(x['q.cfs.ncffoa.t']>0) & (x['q.cfs.ncffia.t']<0) &  (x['q.cfs.ncfffa.t']<0):
        symbol = '奶牛型'
    elif(x['q.cfs.ncffoa.t']>0) & (x['q.cfs.ncffia.t']>0) &  (x['q.cfs.ncfffa.t']<0):
        symbol = '母鸡型'
    elif(x['q.cfs.ncffoa.t']>0) & (x['q.cfs.ncffia.t']<0) &  (x['q.cfs.ncfffa.t']>0):
        symbol = '蛮牛型'
    return symbol
 
df_data['symbol'] = df_data.apply(judgee,axis=1)
df_temp = df_data.pivot(index = 'name', columns = 'year', values = 'symbol')
df_temp.index.name = '公司名称'
df_temp.columns.name = '年份'
df_temp#连续两年以上‘危险’（即经营活动没赚到钱），就要避开

 
#将列名称更改为中文显示，需要注意的是，列名一旦改变，在调用的时候不能再向上面一样调用，需要用中文列名调用
df_data.columns =df_data.columns.map(lambda x: col_map[x])
df_data.to_csv('time_data.csv')


In [5]:
# AKShare获取日际数据

lst_stock = [
    "002195",
    "002223",
    "002912",
    "300033",
    "601998",
    "605398",
    "688590",
    "002987",
    "301236",
    "300662",
]

import akshare as ak

for stock in lst_stock:
    print(stock)
    stock_df = ak.stock_zh_a_hist(
        symbol=stock,
        period="daily",
        start_date="20150430",
        end_date="20250430",
        adjust="",
    )
    stock_df.to_csv(stock + '.csv')
    print(stock_df.head())

002195
           日期    股票代码     开盘     收盘     最高     最低     成交量           成交额    振幅  \
0  2015-04-30  002195  54.57  56.45  58.98  54.53  220419  1.256764e+09  8.15   
1  2015-05-04  002195  55.80  53.69  55.80  52.10  120101  6.450724e+08  6.55   
2  2015-05-05  002195  53.85  52.30  53.85  52.00   84514  4.462262e+08  3.45   
3  2015-05-06  002195  52.42  53.05  55.25  52.42  111473  6.005536e+08  5.41   
4  2015-05-07  002195  52.00  51.28  52.77  51.00   79110  4.103176e+08  3.34   

    涨跌幅   涨跌额    换手率  
0  3.41  1.86  15.01  
1 -4.89 -2.76   8.18  
2 -2.59 -1.39   5.76  
3  1.43  0.75   7.59  
4 -3.34 -1.77   5.39  
002223
           日期    股票代码    开盘     收盘     最高     最低     成交量          成交额    振幅  \
0  2015-04-30  002223  46.5  47.66  47.67  45.90  124311  583582192.0  3.87   
1  2015-05-04  002223  47.5  47.60  48.10  45.99  104055  494037536.0  4.43   
2  2015-05-05  002223  47.4  45.59  48.15  45.52   94625  442258096.0  5.53   
3  2015-05-06  002223  45.7  45.61  47.08  45